In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_df = pd.read_csv("OneDrive/Documents/store-sales-time-series-forecasting/train.csv")
test_df = pd.read_csv("OneDrive/Documents/store-sales-time-series-forecasting/test.csv")
oil_df = pd.read_csv("OneDrive/Documents/store-sales-time-series-forecasting/oil.csv")
store_df = pd.read_csv("OneDrive/Documents/store-sales-time-series-forecasting/stores.csv")
holiday_df = pd.read_csv("OneDrive/Documents/store-sales-time-series-forecasting/holidays_events.csv")
transaction_df = pd.read_csv("OneDrive/Documents/store-sales-time-series-forecasting/transactions.csv")

In [7]:
train_df['is_test'] = 0
test_df['sales'] = np.nan  # since it's not known
test_df['is_test'] = 1

full_df = pd.concat([train_df, test_df], sort=False)
full_df = full_df.sort_values(['store_nbr', 'family', 'date'])  # important!

In [9]:
full_df = full_df.merge(oil_df, on='date', how='left')
full_df = full_df.merge(holiday_df, on='date', how='left')
full_df = full_df.merge(store_df, on='store_nbr', how='left')
full_df = full_df.merge(transaction_df, on=['date', 'store_nbr'], how='left')

In [11]:
print(full_df.head())

     id        date  store_nbr      family  sales  onpromotion  is_test  \
0     0  2013-01-01          1  AUTOMOTIVE    0.0            0        0   
1  1782  2013-01-02          1  AUTOMOTIVE    2.0            0        0   
2  3564  2013-01-03          1  AUTOMOTIVE    3.0            0        0   
3  5346  2013-01-04          1  AUTOMOTIVE    3.0            0        0   
4  7128  2013-01-05          1  AUTOMOTIVE    5.0            0        0   

   dcoilwtico    type_x    locale locale_name              description  \
0         NaN   Holiday  National     Ecuador       Primer dia del ano   
1       93.14       NaN       NaN         NaN                      NaN   
2       92.97       NaN       NaN         NaN                      NaN   
3       93.12       NaN       NaN         NaN                      NaN   
4         NaN  Work Day  National     Ecuador  Recupero puente Navidad   

  transferred   city      state type_y  cluster  transactions  
0       False  Quito  Pichincha      D  

In [13]:
full_df['date'] = pd.to_datetime(full_df['date'])
full_df['day'] = full_df['date'].dt.day
full_df['month'] = full_df['date'].dt.month
full_df['year'] = full_df['date'].dt.year
full_df['dayofweek'] = full_df['date'].dt.dayofweek
full_df['is_weekend'] = full_df['dayofweek'] >= 5
print(full_df.head())

     id       date  store_nbr      family  sales  onpromotion  is_test  \
0     0 2013-01-01          1  AUTOMOTIVE    0.0            0        0   
1  1782 2013-01-02          1  AUTOMOTIVE    2.0            0        0   
2  3564 2013-01-03          1  AUTOMOTIVE    3.0            0        0   
3  5346 2013-01-04          1  AUTOMOTIVE    3.0            0        0   
4  7128 2013-01-05          1  AUTOMOTIVE    5.0            0        0   

   dcoilwtico    type_x    locale  ...   city      state type_y cluster  \
0         NaN   Holiday  National  ...  Quito  Pichincha      D      13   
1       93.14       NaN       NaN  ...  Quito  Pichincha      D      13   
2       92.97       NaN       NaN  ...  Quito  Pichincha      D      13   
3       93.12       NaN       NaN  ...  Quito  Pichincha      D      13   
4         NaN  Work Day  National  ...  Quito  Pichincha      D      13   

  transactions day  month  year  dayofweek  is_weekend  
0          NaN   1      1  2013          1     

In [15]:
print(full_df.isnull().sum())
print(full_df.shape)

id                    0
date                  0
store_nbr             0
family                0
sales             28512
onpromotion           0
is_test               0
dcoilwtico       962280
type_x          2578554
locale          2578554
locale_name     2578554
description     2578554
transferred     2578554
city                  0
state                 0
type_y                0
cluster               0
transactions     277629
day                   0
month                 0
year                  0
dayofweek             0
is_weekend            0
dtype: int64
(3082860, 23)


In [21]:
full_df = full_df.drop(columns=['type_y'])
full_df = full_df.rename(columns={'type_x': 'type'})
print(full_df.head())

     id       date  store_nbr      family  sales  onpromotion  is_test  \
0     0 2013-01-01          1  AUTOMOTIVE    0.0            0        0   
1  1782 2013-01-02          1  AUTOMOTIVE    2.0            0        0   
2  3564 2013-01-03          1  AUTOMOTIVE    3.0            0        0   
3  5346 2013-01-04          1  AUTOMOTIVE    3.0            0        0   
4  7128 2013-01-05          1  AUTOMOTIVE    5.0            0        0   

   dcoilwtico      type    locale  ... transferred   city      state cluster  \
0         NaN   Holiday  National  ...       False  Quito  Pichincha      13   
1       93.14       NaN       NaN  ...         NaN  Quito  Pichincha      13   
2       92.97       NaN       NaN  ...         NaN  Quito  Pichincha      13   
3       93.12       NaN       NaN  ...         NaN  Quito  Pichincha      13   
4         NaN  Work Day  National  ...       False  Quito  Pichincha      13   

  transactions  day  month  year  dayofweek  is_weekend  
0          NaN  

In [27]:
for col in ['dcoilwtico', 'transactions']:
    full_df[col] = full_df.groupby('store_nbr')[col].ffill()


In [33]:
full_df['type'] = full_df['type'].fillna('Work Day')
full_df['locale'] = full_df['locale'].fillna('None')
full_df['locale_name'] = full_df['locale_name'].fillna('None')
full_df['description'] = full_df['description'].fillna('No Event')
full_df['transferred'] = full_df['transferred'].fillna(False)

C:\Users\arjun\AppData\Local\Temp\ipykernel_35540\3268230290.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full_df['transferred'] = full_df['transferred'].fillna(False)


In [35]:
full_df['sales_lag_1'] = full_df.groupby(['store_nbr', 'family'])['sales'].shift(1)
full_df['sales_rollmean_7'] = full_df.groupby(['store_nbr', 'family'])['sales'].shift(1).rolling(7).mean()
train_fe = full_df[full_df['is_test'] == 0].copy()
test_fe = full_df[full_df['is_test'] == 1].copy()

In [43]:
full_df = full_df.sort_values(['store_nbr', 'family', 'date'])
full_df['sales_lag_7'] = full_df.groupby(['store_nbr', 'family'])['sales'].shift(7)
full_df['transactions'] = full_df['transactions'].fillna(0)

C:\Users\arjun\AppData\Local\Temp\ipykernel_35540\4203647148.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  full_df['sales_lag_7'] = full_df.groupby(['store_nbr', 'family'])['sales'].shift(7)


In [49]:
full_df['type'] = full_df['type'].fillna('Work Day')
full_df['transferred'] = full_df['transferred'].fillna(False)
full_df['is_holiday'] = (full_df['type'] != 'Work Day').astype(int)
full_df['is_transferred'] = full_df['transferred'].astype(int)

In [55]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation

features = [
    'store_nbr', 'family', 'onpromotion', 'day', 'month', 'dayofweek',
    'sales_lag_1', 'sales_lag_7', 'sales_rollmean_7',
    'transactions', 'dcoilwtico',
    'is_holiday', 'is_transferred'  
]

for col in ['store_nbr', 'family']:
    full_df[col] = full_df[col].astype('category')
    
train_data = full_df[(full_df['date'] < '2017-07-01') & (full_df['is_test'] == 0)]
valid_data = full_df[(full_df['date'] >= '2017-07-01') & (full_df['is_test'] == 0)]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1621
[LightGBM] [Info] Number of data points in the train set: 2970594, number of used features: 13
[LightGBM] [Info] Start training from score 355.553401
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 46156.2
[200]	valid_0's l2: 41859.8
[300]	valid_0's l2: 40015.9
[400]	valid_0's l2: 38996.4
[500]	valid_0's l2: 38135.5
[600]	valid_0's l2: 37833.9
[700]	valid_0's l2: 37410.2
[800]	valid_0's l2: 36977.4
[900]	valid_0's l2: 36805.5
[1000]	valid_0's l2: 36610
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 36610


LGBMRegressor(learning_rate=0.05, n_estimators=1000, random_state=42)

In [59]:
model = lgb.LGBMRegressor(
    n_estimators=5000,
    learning_rate=0.01,
    random_state=42
)

# Train
model.fit(
    train_data[features], train_data['sales'],
    eval_set=[(valid_data[features], valid_data['sales'])],
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(100)
    ]
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.063729 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1621
[LightGBM] [Info] Number of data points in the train set: 2970594, number of used features: 13
[LightGBM] [Info] Start training from score 355.553401
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 304523
[200]	valid_0's l2: 90745.4
[300]	valid_0's l2: 55781.6
[400]	valid_0's l2: 48372.3
[500]	valid_0's l2: 46181.1
[600]	valid_0's l2: 44971.2
[700]	valid_0's l2: 44007.6
[800]	valid_0's l2: 43288.3
[900]	valid_0's l2: 42510.5
[1000]	valid_0's l2: 42003.4
[1100]	valid_0's l2: 41480.3
[1200]	valid_0's l2: 41027.5
[1300]	valid_0's l2: 40644.1
[1400]	valid_0's l2: 40280.7
[1500]	valid_0's l2: 40062.1
[1600]	valid_0's l2: 39768.6
[1700]	valid_0's l2: 39580.8
[1800]	valid_0's l2: 39353.6
[1900]	valid_0's 

LGBMRegressor(learning_rate=0.01, n_estimators=5000, random_state=42)

In [65]:
test = full_df[full_df['is_test'] == 1].copy()
test['sales'] = model.predict(test[features])
submission = test[['id', 'sales']].copy()
submission['sales'] = submission['sales'].clip(0)  # sales can't be negative
submission.to_csv('forecast_submission.csv', index=False)